In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv('data_assignment2/train_fact.csv')
ext_df = pd.read_csv('data_assignment2/external_data.csv')
prev_df = pd.read_csv('data_assignment2/prev_filtered.csv')

# Feature engineering

Feature engineering is all about creating more information from the information you have. Not all raw data can be used as is.

For example categorical variables cannot be used "as is" therefore we have to one hot encode these variables to use them. Here is one example for gender.


We usually have two types of variables, continuous and categorical

Continuous variables are basically numeric features, it can take any numerical value. In this dataset DAYS BIRTH is an example of a continuous feature. These features are probably a good starting point since we can use them directly in a machine learning model.

Categorical variables are different as in that a variable is categorized in two or more categories. In this dataset GENDER is a categorical variable. It has just three categories: male, female or XNA. Often categorical variables are often strings and have to be transformed in some way as they cannot be used directly in a machine learning model. A machine learning model works with numbers, not with strings.


# The desired state of the data before we can apply machine learning

Before we can train a machine learning we have to transform the dataset into a format that a machine learning model can use.

As mentioned earlier, a machine learning model expects numbers, there should be no string columns.

Q1: how are we going to transform the string columns to a numerical column(s)?

We want to extract as much data from our dataset as possible.

Q2: how can we extract more information from the dataset then the data already provides?

Eventually we want to get to a situation where each column is a feature and the target variable in the end.

Eg:

feat_1, feat_2, feat_3, 'TARGET'


In [4]:
train_data = pd.get_dummies(train_data, columns=['CODE_GENDER'])

In [5]:
train_data.shape

(50000, 124)

Here is an example of how we can add more information such as "income per child" 

In [6]:
train_data['income_per_child'] = train_data['AMT_INCOME_TOTAL'] / (train_data['CNT_CHILDREN'] + 1)

The historical data is more complex, since we only have current records in our prediction table, we have to transform the history to a single row of variables to add to our prediction table.

In [7]:
prev_agg = prev_df.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT': ['min', 'max', 'mean', 'sum']
})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

In [8]:
ext_df = pd.read_csv('data_assignment2/external_data.csv')
ext_agg = ext_df.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': ['min', 'max', 'mean', 'sum']
})
ext_agg.columns = pd.Index(
    ['EXT_' + e[0] + "_" + e[1].upper() for e in ext_agg.columns.tolist()])

We can now merge the aggregates with our prediction table.

In [9]:
train_data = pd.merge(train_data, prev_agg, how='left', on='SK_ID_CURR')
train_data = pd.merge(train_data, ext_agg, how='left', on='SK_ID_CURR')

In [10]:
train_data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CODE_GENDER_XNA,income_per_child,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_AMT_CREDIT_SUM,EXT_AMT_CREDIT_SUM_MIN,EXT_AMT_CREDIT_SUM_MAX,EXT_AMT_CREDIT_SUM_MEAN,EXT_AMT_CREDIT_SUM_SUM
0,100002,1,Cash loans,N,Y,0,202500.000,406597.5,24700.5,351000.0,...,0,202500.000,179055.000,179055.00,179055.000000,1.790550e+05,0.00,450000.000,1.081319e+05,865055.565
1,100003,0,Cash loans,N,N,0,270000.000,1293502.5,35698.5,1129500.0,...,0,270000.000,68053.500,1035882.00,484191.000000,1.452573e+06,22248.00,810000.000,2.543501e+05,1017400.500
2,100004,0,Revolving loans,Y,Y,0,67500.000,135000.0,6750.0,135000.0,...,0,67500.000,20106.000,20106.00,20106.000000,2.010600e+04,94500.00,94537.800,9.451890e+04,189037.800
3,100006,0,Cash loans,N,Y,0,135000.000,312682.5,29686.5,297000.0,...,0,135000.000,0.000,906615.00,291695.500000,2.625260e+06,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,N,Y,0,121500.000,513000.0,21865.5,513000.0,...,0,121500.000,14616.000,284400.00,166638.750000,9.998325e+05,146250.00,146250.000,1.462500e+05,146250.000
5,100008,0,Cash loans,N,Y,0,99000.000,490495.5,27517.5,454500.0,...,0,99000.000,0.000,501975.00,162767.700000,8.138385e+05,95134.50,267606.000,1.561485e+05,468445.500
6,100009,0,Cash loans,Y,Y,1,171000.000,1560726.0,41301.0,1395000.0,...,0,85500.000,38574.000,98239.50,70137.642857,4.909635e+05,35770.50,1777500.000,2.667118e+05,4800811.500
7,100010,0,Cash loans,Y,Y,0,360000.000,1530000.0,42075.0,1530000.0,...,0,360000.000,260811.000,260811.00,260811.000000,2.608110e+05,315000.00,675000.000,4.950000e+05,990000.000
8,100011,0,Cash loans,N,Y,0,112500.000,1019610.0,33826.5,913500.0,...,0,112500.000,0.000,732915.00,261840.375000,1.047362e+06,54000.00,145242.000,1.088071e+05,435228.300
9,100012,0,Revolving loans,N,Y,0,135000.000,405000.0,20250.0,405000.0,...,0,135000.000,0.000,158508.00,74119.500000,2.964780e+05,NaN,NaN,NaN,NaN


In [11]:
train_data.shape

(50000, 133)

You can see we already added 9 new features!

# Your turn!

What other features can you come up with?

- What features have to be transformed, eg categorical variables, transformations over the income etc.
- How can we extract more value from our history?
- What other creative features can you come up with?

Some ideas:

- What's the term of the loan?
- How many times did someone pay late in the past?
- How many times did a previous attempt get rejected?
- Time might be important, more recent information might be more relevant then earlier information